In [ ]:
# =========================
# Fig 5(a) for PINNs with TensorFlow/Keras)
# =========================
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers
import matplotlib.pyplot as plt
import random, math

# Use float64 for stable higher-order derivatives
tf.keras.backend.set_floatx('float64')

# -------- Problem setup --------
v = 50.0   # advection (Case v= 1.0 works well)
k = 1.0    # diffusion

def u_true(x, v=v, k=k):
    # analytic solution of k u'' - v u' = 0 with u(0)=0, u(1)=1
    return (np.exp((v/k)*x) - 1.0) / (math.e**(v/k) - 1.0)

# Dense grid for MSE
x_eval = np.linspace(0.0, 1.0, 2001).reshape(-1,1)
u_eval = u_true(x_eval)

# Collocation + BC points
N_c = 2000
N_b = 200  # half at x=0, half at x=1
# N_c = 100
# N_b = 10  # half at x=0, half at x=1

rng = np.random.default_rng(123)
x_c_np = rng.random((N_c,1))           # uniform in (0,1)
x_b0_np = np.zeros((N_b//2,1))
x_b1_np = np.ones((N_b//2,1))
x_c  = tf.convert_to_tensor(x_c_np)
x_b0 = tf.convert_to_tensor(x_b0_np)
x_b1 = tf.convert_to_tensor(x_b1_np)

# -------- Model builder --------
class Sin(layers.Layer):
    def call(self, z): return tf.sin(z)

def build_pinn(activation='tanh', init='glorot_uniform'):
    if activation == 'tanh':
        act = layers.Activation('tanh')
    elif activation == 'sigmoid':
        act = layers.Activation('sigmoid')
    elif activation == 'sin':
        act = Sin()
    else:
        raise ValueError("activation must be 'tanh', 'sigmoid', or 'sin'")

    if init == 'glorot_uniform':
        kinit = initializers.GlorotUniform()
    elif init == 'he_uniform':
        kinit = initializers.HeUniform()
    else:
        raise ValueError("init must be 'glorot_uniform' or 'he_uniform'")

    NN = keras.Sequential([
        layers.Input((1,), dtype=tf.float64),
        layers.Dense(32, activation=None, kernel_initializer=kinit, bias_initializer='zeros', dtype=tf.float64),
        act,
        layers.Dense(10, activation=None, kernel_initializer=kinit, bias_initializer='zeros', dtype=tf.float64),
        act,
        layers.Dense(10, activation=None, kernel_initializer=kinit, bias_initializer='zeros', dtype=tf.float64),
        act,
        layers.Dense(10, activation=None, kernel_initializer=kinit, bias_initializer='zeros', dtype=tf.float64),
        act,
        layers.Dense(1,  activation=None, kernel_initializer=kinit, bias_initializer='zeros', dtype=tf.float64),
    ])
    return NN

# -------- PINN losses (PDE + BC) --------
@tf.function
def pde_residual(model, x):
    x = tf.cast(x, tf.float64)
    with tf.GradientTape(persistent=True) as t2:
        t2.watch(x)
        with tf.GradientTape() as t1:
            t1.watch(x)
            u = model(x)                # (N,1)
        u_x = t1.gradient(u, x)         # (N,1)
    u_xx = t2.gradient(u_x, x)          # (N,1)
    del t2
    return v * u_x - k * u_xx           # residual: v u' - k u''

def pinn_loss(model, x_c, x_b0, x_b1, lam=1.0):
    r = pde_residual(model, x_c)
    L_pde = tf.reduce_mean(tf.square(r))
    u_b0 = model(x_b0)
    u_b1 = model(x_b1)
    L_bc = tf.reduce_mean(tf.square(u_b0)) + tf.reduce_mean(tf.square(u_b1 - 1.0))
    return (L_pde / lam) + L_bc, L_pde, L_bc

# -------- One training run --------
def train_one(model, steps=20000, lr=5e-3, lam=1.0, patience=500, min_lr=1e-6, verbose=False):
    opt = keras.optimizers.Adam(learning_rate=lr)
    best = np.inf
    plateau = 0
    for it in range(steps):
        with tf.GradientTape() as tape:
            loss, Lp, Lb = pinn_loss(model, x_c, x_b0, x_b1, lam=lam)
        grads = tape.gradient(loss, model.trainable_variables)
        opt.apply_gradients(zip(grads, model.trainable_variables))

        # reduce-on-plateau every 200 iters
        if (it+1) % 200 == 0:
            cur = float(loss.numpy())
            if verbose: print(f"it {it+1:5d}  loss={cur:.3e}  Lp={float(Lp.numpy()):.3e}  Lb={float(Lb.numpy()):.3e}  lr={opt.learning_rate.numpy():.2e}")
            if cur < best - 1e-9:
                best = cur; plateau = 0
            else:
                plateau += 1
                if plateau >= patience//200 and opt.learning_rate.numpy() > min_lr + 1e-12:
                    opt.learning_rate.assign(max(opt.learning_rate.numpy()*0.5, min_lr))
                    plateau = 0

    # MSE on dense grid
    u_hat = model(tf.convert_to_tensor(x_eval))
    mse = float(tf.reduce_mean(tf.square(u_hat - u_eval)).numpy())
    return mse

# -------- Run multiple seeds per config --------
def set_seed(s):
    random.seed(s); np.random.seed(s); tf.random.set_seed(s)

def run_config(label, activation, init, n_runs=20, steps=20000, lr=5e-3, lam=1.0):
    mses = []
    for i in range(n_runs):
        set_seed(1000 + i)
        model = build_pinn(activation=activation, init=init)
        mse = train_one(model, steps=steps, lr=lr, lam=lam)
        mses.append(mse)
    print(f"{label}: median MSE = {np.median(mses):.3e}")
    return mses

# -------- Define configurations (as in Fig. 5(a) for PINNs) --------
configs = [
    ("Std-Xav-Tanh",    'tanh',    'glorot_uniform'),
    ("Std-He-Tanh",     'tanh',    'he_uniform'),
    ("Std-Xav-Sigmoid", 'sigmoid', 'glorot_uniform'),
    ("Std-He-Sigmoid",  'sigmoid', 'he_uniform'),
    ("Std-Xav-Sin",     'sin',     'glorot_uniform'),
    ("Std-He-Sin",      'sin',     'he_uniform'),
]

labels, data = [], []
for name, act, init in configs:
    mses = run_config(name, act, init, n_runs=20, steps=50000, lr=5e-3, lam=1.0)
    labels.append(name); data.append(mses)

# -------- Boxplot (log scale) --------
plt.figure(figsize=(10,4))
plt.boxplot(data, showfliers=False)
plt.yscale('log')
plt.xticks(range(1, len(labels)+1), labels, rotation=45, ha='right')
plt.ylabel('Solution MSE (log scale)')
plt.title(f'1D steady convection–diffusion (v={v:g}, k={k:g}): PINN MSE across 20 runs')
plt.tight_layout()
plt.show()
plt.savefig("fig5a_pinns.png", dpi=300, bbox_inches='tight')
#plt.close()

import pickle

# Save
with open("results.pkl", "wb") as f:
    pickle.dump({"labels": labels, "data": data}, f)

